In [1]:
import os
os.chdir("../")
%pwd

'c:\\Users\\abhis\\Desktop\\MLProjects\\Movie Recommender'

In [3]:
import pandas as pd

movie_df = pd.read_csv('artifacts/data_ingestion/ml-latest/movies.csv')
movie_df.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [10]:
links_df = pd.read_csv('artifacts/data_ingestion/ml-latest/links.csv')
links_df.head()

,movieId,imdbId,tmdbId
0,1,114709,862.0
1,2,113497,8844.0
2,3,113228,15602.0
3,4,114885,31357.0
4,5,113041,11862.0


In [13]:
ratings_df = pd.read_csv('artifacts/data_ingestion/ml-latest/ratings.csv')
ratings_df.head()

,userId,movieId,rating,timestamp
0,1,307,3.5,1256677221
1,1,481,3.5,1256677456
2,1,1091,1.5,1256677471
3,1,1257,4.5,1256677460
4,1,1449,4.5,1256677264


In [14]:
tags_df = pd.read_csv('artifacts/data_ingestion/ml-latest/tags.csv')
tags_df.head()

,userId,movieId,tag,timestamp
0,14,110,epic,1443148538
1,14,110,Medieval,1443148532
2,14,260,sci-fi,1442169410
3,14,260,space action,1442169421
4,14,318,imdb top 250,1442615195


In [21]:
genome_scores_df = pd.read_csv('artifacts/data_ingestion/ml-latest/genome-scores.csv')
genome_scores_df.head(5)

,movieId,tagId,relevance
0,1,1,0.02900
1,1,2,0.02375
2,1,3,0.05425
3,1,4,0.06875
4,1,5,0.16000


In [19]:
genome_tags_df = pd.read_csv('artifacts/data_ingestion/ml-latest/genome-tags.csv')
genome_tags_df.head(20)

,tagId,tag
0,1,007
1,2,007 (series)
2,3,18th century
3,4,1920s
4,5,1930s
5,6,1950s
6,7,1960s
7,8,1970s
8,9,1980s
9,10,19th century


In [11]:
links_df_small = links_df[:50]

In [6]:
# import requests

import json
import urllib3

API_KEY = os.environ.get('TMDB_API_KEY')


def get_movie_details(movie_id):
    movie_url = f'https://api.themoviedb.org/3/movie/{movie_id}?api_key={API_KEY}'
    response = requests.get(movie_url)
    if response.status_code == 200:
        movie_data = response.json()

        # Extracting the desired fields
        genres = [genre['name'] for genre in movie_data['genres']]
        overview = movie_data['overview']
        popularity = movie_data['popularity']
        poster_path = movie_data['poster_path']
        vote_average = movie_data['vote_average']
        vote_count = movie_data['vote_count']

        # Extracting the director's name (assuming it is the first crew member with job "Director")
        credits_url = f'https://api.themoviedb.org/3/movie/{movie_id}/credits?api_key={API_KEY}'
        credits_response = requests.get(credits_url)
        if credits_response.status_code == 200:
            credits_data = credits_response.json()
            crew = credits_data['crew']
            director = next((member['name'] for member in crew if member['job'] == 'Director'), None)
        else:
            director = None

        # Requesting the keywords endpoint
        keywords_url = f'https://api.themoviedb.org/3/movie/{movie_id}/keywords?api_key={API_KEY}'
        keywords_response = requests.get(keywords_url)
        if keywords_response.status_code == 200:
            keywords_data = keywords_response.json()
            keywords = [keyword['name'] for keyword in keywords_data['keywords']]
        else:
            keywords = []
        data =  {"genres" : genres,"overview" : overview,"popularity" : popularity,\
                    "poster_path":poster_path,"vote_average" : vote_average,"vote_count" : vote_count,\
                    "director" : director,"keywords" : keywords}
        
        return data
    else:
        print('Error occurred while requesting data:', response.status_code)


links_df_small = links_df_small.merge(links_df_small['tmdbId'].apply(lambda x: pd.Series(get_movie_details(x))), left_index=True, right_index=True)

print(links_df_small.head(2))

   movieId  imdbId  tmdbId                                  genres  \
0        1  114709   862.0  [Animation, Adventure, Family, Comedy]   
1        2  113497  8844.0            [Adventure, Fantasy, Family]   

                                            overview  popularity  \
0  Led by Woody, Andy's toys live happily in his ...     101.402   
1  When siblings Judy and Peter discover an encha...      16.794   

                        poster_path  vote_average  vote_count       director  \
0  /uXDfjJbdP4ijW5hWSBrPrlKpxab.jpg         8.000       16771  John Lasseter   
1  /v2XHtmVqpERPy0HA1y9wltoeEgW.jpg         7.238        9636   Joe Johnston   

                                            keywords  
0  [martial arts, jealousy, friendship, bullying,...  
1  [giant insect, board game, jungle, disappearan...  


In [8]:
# using urllib

import os
import json
import urllib3

API_KEY = os.environ.get('TMDB_API_KEY')


def get_movie_details(movie_id):
    movie_url = f'https://api.themoviedb.org/3/movie/{movie_id}?api_key={API_KEY}'

    http = urllib3.PoolManager()
    response = http.request('GET', movie_url)

    if response.status == 200:
        movie_data = json.loads(response.data.decode('utf-8'))

        # Extracting the desired fields
        genres = [genre['name'] for genre in movie_data['genres']]
        overview = movie_data['overview']
        popularity = movie_data['popularity']
        poster_path = movie_data['poster_path']
        vote_average = movie_data['vote_average']
        vote_count = movie_data['vote_count']
    else:
        print('Error occurred while requesting data:', response.status)

    # Extracting the director's name (assuming it is the first crew member with job "Director")
    credits_url = f'https://api.themoviedb.org/3/movie/{movie_id}/credits?api_key={API_KEY}'
    credits_response = http.request('GET', credits_url)

    if credits_response.status == 200:
        credits_data = json.loads(credits_response.data.decode('utf-8'))
        crew = credits_data['crew']
        director = next((member['name'] for member in crew if member['job'] == 'Director'), None)
    else:
        director = None

    # Requesting the keywords endpoint
    keywords_url = f'https://api.themoviedb.org/3/movie/{movie_id}/keywords?api_key={API_KEY}'
    keywords_response = http.request('GET', keywords_url)

    if keywords_response.status == 200:
        keywords_data = json.loads(keywords_response.data.decode('utf-8'))
        keywords = [keyword['name'] for keyword in keywords_data['keywords']]
    else:
        keywords = []

    data = {
        "genres": genres,
        "overview": overview,
        "popularity": popularity,
        "poster_path": poster_path,
        "vote_average": vote_average,
        "vote_count": vote_count,
        "director": director,
        "keywords": keywords
    }

    return data
    



In [9]:
get_movie_details(8844)

{'genres': ['Adventure', 'Fantasy', 'Family'],
 'overview': "When siblings Judy and Peter discover an enchanted board game that opens the door to a magical world, they unwittingly invite Alan -- an adult who's been trapped inside the game for 26 years -- into their living room. Alan's only hope for freedom is to finish the game, which proves risky as all three find themselves running from giant rhinoceroses, evil monkeys and other terrifying creatures.",
 'popularity': 16.794,
 'poster_path': '/v2XHtmVqpERPy0HA1y9wltoeEgW.jpg',
 'vote_average': 7.238,
 'vote_count': 9636,
 'director': 'Joe Johnston',
 'keywords': ['giant insect',
  'board game',
  'jungle',
  'disappearance',
  'recluse',
  'stampede',
  'based on young adult novel']}

In [12]:
links_df_small = links_df_small.merge(links_df_small['tmdbId'].apply(lambda x: pd.Series(get_movie_details(x))), left_index=True, right_index=True)

print(links_df_small.head(2))

   movieId  imdbId  tmdbId                                  genres  \
0        1  114709   862.0  [Animation, Adventure, Family, Comedy]   
1        2  113497  8844.0            [Adventure, Fantasy, Family]   

                                            overview  popularity  \
0  Led by Woody, Andy's toys live happily in his ...     101.402   
1  When siblings Judy and Peter discover an encha...      16.794   

                        poster_path  vote_average  vote_count       director  \
0  /uXDfjJbdP4ijW5hWSBrPrlKpxab.jpg         8.000       16771  John Lasseter   
1  /v2XHtmVqpERPy0HA1y9wltoeEgW.jpg         7.238        9636   Joe Johnston   

                                            keywords  
0  [martial arts, jealousy, friendship, bullying,...  
1  [giant insect, board game, jungle, disappearan...  


In [2]:
import os
from pathlib import Path
Path(os.path.join('artifacts/data_ingestion', 'ml-latest/movies.csv'))

WindowsPath('artifacts/data_ingestion/ml-latest/movies.csv')

In [4]:
 os.path.join('artifacts/data_ingestion', f'1.json')

'artifacts/data_ingestion\\1.json'

In [6]:
import pandas as pd
import numpy as np

# Define the length of the dataframes
length = 5

# Define column names for the first dataframe
columns1 = ['Column A', 'Column B', 'Column C']

# Define column names for the second dataframe
columns2 = ['Column X', 'Column Y', 'Column Z']

# Create the first dataframe with random values
df1 = pd.DataFrame(np.random.randn(length, len(columns1)), columns=columns1)

# Create the second dataframe with random values
df2 = pd.DataFrame(np.random.randn(length, len(columns2)), columns=columns2)

# Print the dataframes
print(df1)
print(df2)


   Column A  Column B  Column C
0 -0.396550  1.172845 -1.075174
1  0.180934  0.699182  0.138170
2 -0.229210 -0.463325 -0.335848
3 -0.593965 -0.767824 -0.165208
4  0.366052 -1.709240 -0.460423
   Column X  Column Y  Column Z
0  0.690416 -0.276372  0.057476
1  0.091253  0.714823 -0.708617
2 -0.359204  1.536398 -0.463248
3  0.678518  0.050205  2.394477
4 -1.649797  1.288121  0.710626


In [8]:
merged_df = pd.merge(df1, df2, left_index=True, right_index=True)

In [9]:
merged_df

,Column A,Column B,Column C,Column X,Column Y,Column Z
0,-0.396550,1.172845,-1.075174,0.690416,-0.276372,0.057476
1,0.180934,0.699182,0.138170,0.091253,0.714823,-0.708617
2,-0.229210,-0.463325,-0.335848,-0.359204,1.536398,-0.463248
3,-0.593965,-0.767824,-0.165208,0.678518,0.050205,2.394477
4,0.366052,-1.709240,-0.460423,-1.649797,1.288121,0.710626
